<a href="https://colab.research.google.com/github/Diz138/2022NFLScoutingCombineAnalysis/blob/main/modeling/in_game_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy import stats
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import math

# path to go, in, out routes cleaned tracking data in github
#go_path = 'in_game/data/go_routes.csv'
#in_path = 'in_game/data/in_routes.csv'
#out_path = 'in_game/data/out_route.csv'

in_path = r"C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\in_game\data\in_routes.csv"
out_path = r"C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\in_game\data\out_routes.csv"
go_path = r'C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\in_game\data\go_routes.csv'

go_df = pd.read_csv(go_path)
in_df = pd.read_csv(in_path)
out_df = pd.read_csv(out_path)

In [2]:
print(go_df.playerId.unique())

[54476 54517 54475 54481 54519 54499 54628 54483 54508 54564 54613 54570
 54553 54515 54667 54536 54656]


In [3]:
def metric_creation(dataframe, start_pos, end_pos):
    """
    Function which provides the metrics for straight segments of a specified length.
    """
    start = dataframe.head(1)
    total_velX = start.inverted_vel_x.iloc[0]
    total_velY = start.inverted_vel_y.iloc[0]
    total_accX = start.inverted_acc_x.iloc[0]
    total_accY = start.inverted_acc_y.iloc[0]
    max_vel = 0
    pos_vel = 0
    max_acc = 0
    pos_acc = 0
    time = 0
    stopper = 0
    max_vel = -1000000
    max_acc = -1000000
    mean_var_list = []
    for i in range(1,len(dataframe)):
        starting = start.inverted_x.iloc[0] + start_pos
        ending = start.inverted_x.iloc[0] + end_pos
        if dataframe.inverted_x.iloc[i] >= starting and dataframe.inverted_x.iloc[i] <= ending:
            mean_var_list.append(dataframe.inverted_y.iloc[i])
            total_velX = total_velX + dataframe.inverted_vel_x.iloc[i]
            total_velY = total_velY + dataframe.inverted_vel_y.iloc[i]
            total_accX = total_accX + dataframe.inverted_acc_x.iloc[i]
            total_accY = total_accY + dataframe.inverted_acc_y.iloc[i]
            time += 1
            if dataframe.inverted_vel_x.iloc[i] > max_vel:
                max_vel = dataframe.inverted_vel_x.iloc[i]
                pos_vel = dataframe.inverted_x.iloc[i] - start_pos
            if dataframe.inverted_acc_x.iloc[i] > max_acc:
                max_acc = dataframe.inverted_acc_x.iloc[i]
                pos_acc = dataframe.inverted_x.iloc[i] - start_pos 
    mean_y = sum(mean_var_list)/len(mean_var_list) if len(mean_var_list) > 0 else 0
    variance_y = sum((i - mean_y) ** 2 for i in mean_var_list) / len(mean_var_list) if len(mean_var_list) > 0 else 0
    return total_velX/time, total_accX/time, time/10, max_vel, pos_vel, max_acc, pos_acc, mean_y, variance_y

In [4]:
def get_normal_percentile_90(data):
  """
  Function which provides the 90th percentile metric for a sample of metrics with a normal distribution.
  """
  if len(data.drop_duplicates().tolist()) > 1:
    mu, std = stats.norm.fit(data)
    percentile = stats.norm.ppf(0.90, loc=mu, scale=std)
  else:
    mu = data.drop_duplicates().tolist()[0]
    std = 0.1
    percentile = stats.norm.ppf(0.90, loc=mu, scale=std)
  return percentile

def get_normal_percentile_95(data):
  """
  Function which provides the 95th percentile metric for a sample of metrics with a normal distribution.
  """
  if len(data.drop_duplicates().tolist()) > 1:
    #for val in data:
        #print(type(val))
    data = pd.to_numeric(data)
    mu, std = stats.norm.fit(data)
    percentile = stats.norm.ppf(0.95, loc=mu, scale=std)
  elif len(data) > 0:
    mu = data.drop_duplicates().tolist()[0]
    std = 0.1
    percentile = stats.norm.ppf(0.95, loc=mu, scale=std)
  else: percentile = None
  return percentile

def get_normal_percentile_5(data):
  """
  Function which provides the 5th percentile metric for a sample of metrics with a normal distribution.
  """
  if len(data.drop_duplicates().tolist()) > 1:
    mu, std = stats.norm.fit(data)
    percentile = stats.norm.ppf(0.5, loc=mu, scale=std)
  elif len(data) > 0:
    mu = data.drop_duplicates().tolist()[0]
    std = 0.1
    percentile = stats.norm.ppf(0.5, loc=mu, scale=std)
  else: percentile = None
  return percentile

def get_percentile_95(data):
  """
  Function which provides the 90th percentile metric for a population of metrics.
  """
  metrics = np.array(data)
  percentile = np.percentile(metrics, 95)
  return percentile

def get_percentile_90(data):
  """
  Function which provides the 90th percentile metric for a population of metrics.
  """
  metrics = np.array(data)
  percentile = np.percentile(metrics, 90)
  return percentile

def percentile_metrics(df, columns):
    """
    Function which provides the 95th percentile metric for all the metrics in a dataframe.
    """
    df = df[columns]
    metric_list =[]
    # calculate 95th percentile of all columns except playerId and route
    #percentile = grouped.apply(lambda x: x.iloc[:, 2:].quantile(0.95))
    percentile_metrics_df = pd.DataFrame(columns=columns)
    metric_list.append(df.player_id.drop_duplicates().tolist()[0])
    metric_list.append(df.route.drop_duplicates().tolist()[0])
    for metric in columns[2:]:
        if 'time' in metric:
            curr_metric = get_normal_percentile_5(df[metric].dropna())
            metric_list.append(curr_metric)
        else:
            print(metric)
            curr_metric = get_normal_percentile_95(df[metric].dropna())
            metric_list.append(curr_metric)
    percentile_metrics_df.loc[len(percentile_metrics_df)] = metric_list
    return percentile_metrics_df
    #return percentile

# GO Route Metric Creation

In [5]:
def go_metric_creation(df):
  playIds = df.playId.drop_duplicates().tolist()
  metrics_Go = pd.DataFrame(columns = ['player_id','play_id','game_key','route','vel_x_2_5', 'acc_x_2_5', 'time_2_5', 'max_vel_2_5', 'pos_vel_2_5', 'max_acc_2_5', 'pos_acc_2_5', 'mean_y_2_5', 'variance_y_2_5','vel_x_5', 'acc_x_5', 'time_5', 'max_vel_5', 'pos_vel_5', 'max_acc_5', 'pos_acc_5', 'mean_y_5', 'variance_y_5','vel_x_5_10', 'acc_x_5_10', 'time_5_10', 'max_vel_5_10', 'pos_vel_5_10', 'max_acc_5_10', 'pos_acc_5_10', 'mean_y_5_10', 'variance_y_5_10','vel_x_10', 'acc_x_10', 'time_10', 'max_vel_10', 'pos_vel_10', 'max_acc_10', 'pos_acc_10', 'mean_y_10', 'variance_y_10','vel_x_10_15', 'acc_x_10_15', 'time_10_15', 'max_vel_10_15', 'pos_vel_10_15', 'max_acc_10_15', 'pos_acc_10_15', 'mean_y_10_15', 'variance_y_10_15','vel_x_15_20', 'acc_x_15_20', 'time_15_20', 'max_vel_15_20', 'pos_vel_15_20', 'max_acc_15_20', 'pos_acc_15_20', 'mean_y_15_20', 'variance_y_15_20','vel_x_10_20', 'acc_x_10_20', 'time_10_20', 'max_vel_10_20', 'pos_vel_10_20', 'max_acc_10_20', 'pos_acc_10_20', 'mean_y_10_20', 'variance_y_10_20','vel_x_20_25', 'acc_x_20_25', 'time_20_25', 'max_vel_20_25', 'pos_vel_20_25', 'max_acc_20_25', 'pos_acc_20_25', 'mean_y_20_25', 'variance_y_20_25','vel_x_25_30', 'acc_x_25_30', 'time_25_30', 'max_vel_25_30', 'pos_vel_25_30', 'max_acc_25_30', 'pos_acc_25_30', 'mean_y_25_30', 'variance_y_25_30','vel_x_20_30', 'acc_x_20_30', 'time_20_30', 'max_vel_20_30', 'pos_vel_20_30', 'max_acc_20_30', 'pos_acc_20_30', 'mean_y_20_30', 'variance_y_20_30','vel_x_30_35', 'acc_x_30_35', 'time_30_35', 'max_vel_30_35', 'pos_vel_30_35', 'max_acc_30_35', 'pos_acc_30_35', 'mean_y_30_35', 'variance_y_30_35','vel_x_30_40', 'acc_x_30_40', 'time_30_40', 'max_vel_30_40', 'pos_vel_30_40', 'max_acc_30_40', 'pos_acc_30_40', 'mean_y_30_40', 'variance_y_30_40','vel_x_35_40', 'acc_x_35_40', 'time_35_40', 'max_vel_35_40', 'pos_vel_35_40', 'max_acc_35_40', 'pos_acc_35_40', 'mean_y_35_40', 'variance_y_35_40'])
  i = 0
  for play in playIds:
    play_df = df.loc[df.playId == play]
    subtraction = play_df.inverted_x.iloc[len(play_df)-1] - play_df.inverted_x.iloc[0]
    if subtraction < 2.5:
        metrics = [play_df.playerId.iloc[0],play_df.playId.iloc[0],play_df.gameKey.iloc[0],'GO']
        for i in range(117):
            metrics.append(0)
    if subtraction >= 2.5:
        #buckets["2.5"].append(key)
        vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5 = metric_creation(play_df, 0, 2.5)
        vel_x_5 = acc_x_5 = time_5 = max_vel_5 = pos_vel_5 = max_acc_5 = pos_acc_5 = mean_y_5 = variance_y_5 = vel_x_5_10 = acc_x_5_10 = time_5_10 = max_vel_5_10 = pos_vel_5_10 = max_acc_5_10 = pos_acc_5_10 = mean_y_5_10 = variance_y_5_10 =vel_x_10 = acc_x_10 = time_10 = max_vel_10 = pos_vel_10 = max_acc_10 = pos_acc_10 = mean_y_10 = variance_y_10 = vel_x_10_15 = acc_x_10_15 = time_10_15 = max_vel_10_15 = pos_vel_10_15 = max_acc_10_15=  pos_acc_10_15 = mean_y_10_15 = variance_y_10_15 =vel_x_15_20 = acc_x_15_20 = time_15_20 = max_vel_15_20 = pos_vel_15_20 = max_acc_15_20 = pos_acc_15_20 = mean_y_15_20 =variance_y_15_20 = vel_x_10_20 = acc_x_10_20 = time_10_20 = max_vel_10_20 = pos_vel_10_20 = max_acc_10_20 = pos_acc_10_20 = mean_y_10_20 = variance_y_10_20 =vel_x_20_25 = acc_x_20_25 = time_20_25 = max_vel_20_25 = pos_vel_20_25 = max_acc_20_25 = pos_acc_20_25 = mean_y_20_25 = variance_y_20_25 = vel_x_25_30 = acc_x_25_30 = time_25_30 = max_vel_25_30 = pos_vel_25_30 = max_acc_25_30 = pos_acc_25_30 = mean_y_25_30 = variance_y_25_30 = vel_x_20_30 = acc_x_20_30 = time_20_30 = max_vel_20_30 = pos_vel_20_30 = max_acc_20_30 = pos_acc_20_30 = mean_y_20_30 = variance_y_20_30 = vel_x_30_35= acc_x_30_35 = time_30_35 = max_vel_30_35= pos_vel_30_35= max_acc_30_35= pos_acc_30_35 = mean_y_30_35 = variance_y_30_35 = vel_x_30_40 = acc_x_30_40 = time_30_40 = max_vel_30_40 = pos_vel_30_40 = max_acc_30_40 = pos_acc_30_40 = mean_y_30_40 = variance_y_30_40 =vel_x_35_40 =acc_x_35_40 =time_35_40 =max_vel_35_40 =pos_vel_35_40 =max_acc_35_40 =pos_acc_35_40= mean_y_35_40 =variance_y_35_40 = None
        player_id = play_df.playerId.iloc[0]
        if player_id == 0.0:
            print(f"yards {subtraction} and val {player_id}")
        play_id = play 
        game_key = play_df.gameKey.iloc[0] 
        route = play_df.route.iloc[0] 
        metrics = [player_id,play_id,game_key,route,vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5,vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5,vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10,vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10,vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15,vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20,vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20,vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25,vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30,vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30,vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35,vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40,vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40]

    if subtraction >= 5:
        #buckets["5"].append(key)
        vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5 = metric_creation(play_df, 0, 2.5)
        vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5 = metric_creation(play_df,0,5)
        vel_x_5_10 = acc_x_5_10 = time_5_10 = max_vel_5_10 = pos_vel_5_10 = max_acc_5_10 = pos_acc_5_10 = mean_y_5_10 = variance_y_5_10 =vel_x_10 = acc_x_10 = time_10 = max_vel_10 = pos_vel_10 = max_acc_10 = pos_acc_10 = mean_y_10 = variance_y_10 = vel_x_10_15 = acc_x_10_15 = time_10_15 = max_vel_10_15 = pos_vel_10_15 = max_acc_10_15=  pos_acc_10_15 = mean_y_10_15 = variance_y_10_15 =vel_x_15_20 = acc_x_15_20 = time_15_20 = max_vel_15_20 = pos_vel_15_20 = max_acc_15_20 = pos_acc_15_20 = mean_y_15_20 =variance_y_15_20 = vel_x_10_20 = acc_x_10_20 = time_10_20 = max_vel_10_20 = pos_vel_10_20 = max_acc_10_20 = pos_acc_10_20 = mean_y_10_20 = variance_y_10_20 =vel_x_20_25 = acc_x_20_25 = time_20_25 = max_vel_20_25 = pos_vel_20_25 = max_acc_20_25 = pos_acc_20_25 = mean_y_20_25 = variance_y_20_25 = vel_x_25_30 = acc_x_25_30 = time_25_30 = max_vel_25_30 = pos_vel_25_30 = max_acc_25_30 = pos_acc_25_30 = mean_y_25_30 = variance_y_25_30 = vel_x_20_30 = acc_x_20_30 = time_20_30 = max_vel_20_30 = pos_vel_20_30 = max_acc_20_30 = pos_acc_20_30 = mean_y_20_30 = variance_y_20_30 = vel_x_30_35= acc_x_30_35 = time_30_35 = max_vel_30_35= pos_vel_30_35= max_acc_30_35= pos_acc_30_35 = mean_y_30_35 = variance_y_30_35 = vel_x_30_40 = acc_x_30_40 = time_30_40 = max_vel_30_40 = pos_vel_30_40 = max_acc_30_40 = pos_acc_30_40 = mean_y_30_40 = variance_y_30_40 =vel_x_35_40 =acc_x_35_40 =time_35_40 =max_vel_35_40 =pos_vel_35_40 =max_acc_35_40 =pos_acc_35_40= mean_y_35_40 =variance_y_35_40 = None
        player_id = play_df.playerId.iloc[0]
        if player_id == 0.0:
            print(f"yards {subtraction} and val {player_id}")
        play_id = play 
        game_key = play_df.gameKey.iloc[0] 
        route = play_df.route.iloc[0] 
        metrics = [player_id,play_id,game_key,route,vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5,vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5,vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10,vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10,vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15,vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20,vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20,vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25,vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30,vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30,vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35,vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40,vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40]

    if subtraction >= 10:
        #buckets["10"].append(key)
        vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5 = metric_creation(play_df, 0, 2.5)
        vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5 = metric_creation(play_df,0,5)
        vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10 = metric_creation(play_df,5,10)
        vel_x_10 = acc_x_10 = time_10 = max_vel_10 = pos_vel_10 = max_acc_10 = pos_acc_10 = mean_y_10 = variance_y_10 = vel_x_10_15 = acc_x_10_15 = time_10_15 = max_vel_10_15 = pos_vel_10_15 = max_acc_10_15=  pos_acc_10_15 = mean_y_10_15 = variance_y_10_15 =vel_x_15_20 = acc_x_15_20 = time_15_20 = max_vel_15_20 = pos_vel_15_20 = max_acc_15_20 = pos_acc_15_20 = mean_y_15_20 =variance_y_15_20 = vel_x_10_20 = acc_x_10_20 = time_10_20 = max_vel_10_20 = pos_vel_10_20 = max_acc_10_20 = pos_acc_10_20 = mean_y_10_20 = variance_y_10_20 =vel_x_20_25 = acc_x_20_25 = time_20_25 = max_vel_20_25 = pos_vel_20_25 = max_acc_20_25 = pos_acc_20_25 = mean_y_20_25 = variance_y_20_25 = vel_x_25_30 = acc_x_25_30 = time_25_30 = max_vel_25_30 = pos_vel_25_30 = max_acc_25_30 = pos_acc_25_30 = mean_y_25_30 = variance_y_25_30 = vel_x_20_30 = acc_x_20_30 = time_20_30 = max_vel_20_30 = pos_vel_20_30 = max_acc_20_30 = pos_acc_20_30 = mean_y_20_30 = variance_y_20_30 = vel_x_30_35= acc_x_30_35 = time_30_35 = max_vel_30_35= pos_vel_30_35= max_acc_30_35= pos_acc_30_35 = mean_y_30_35 = variance_y_30_35 = vel_x_30_40 = acc_x_30_40 = time_30_40 = max_vel_30_40 = pos_vel_30_40 = max_acc_30_40 = pos_acc_30_40 = mean_y_30_40 = variance_y_30_40 =vel_x_35_40 =acc_x_35_40 =time_35_40 =max_vel_35_40 =pos_vel_35_40 =max_acc_35_40 =pos_acc_35_40= mean_y_35_40 =variance_y_35_40 = None
        player_id = play_df.playerId.iloc[0]
        if player_id == 0.0:
            print(f"yards {subtraction} and val {player_id}")
        play_id = play 
        game_key = play_df.gameKey.iloc[0] 
        route = play_df.route.iloc[0] 
        metrics = [player_id,play_id,game_key,route,vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5,vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5,vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10,vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10,vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15,vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20,vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20,vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25,vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30,vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30,vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35,vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40,vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40]


    if subtraction >= 15:
        #buckets["15"].append(key)
        vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5 = metric_creation(play_df, 0, 2.5)
        vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5 = metric_creation(play_df,0,5)
        vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10 = metric_creation(play_df,5,10)
        vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10 = metric_creation(play_df,0,10)
        vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15 = metric_creation(play_df,10,15)
        vel_x_15_20 = acc_x_15_20 = time_15_20 = max_vel_15_20 = pos_vel_15_20 = max_acc_15_20 = pos_acc_15_20 = mean_y_15_20 =variance_y_15_20 = vel_x_10_20 = acc_x_10_20 = time_10_20 = max_vel_10_20 = pos_vel_10_20 = max_acc_10_20 = pos_acc_10_20 = mean_y_10_20 = variance_y_10_20 =vel_x_20_25 = acc_x_20_25 = time_20_25 = max_vel_20_25 = pos_vel_20_25 = max_acc_20_25 = pos_acc_20_25 = mean_y_20_25 = variance_y_20_25 = vel_x_25_30 = acc_x_25_30 = time_25_30 = max_vel_25_30 = pos_vel_25_30 = max_acc_25_30 = pos_acc_25_30 = mean_y_25_30 = variance_y_25_30 = vel_x_20_30 = acc_x_20_30 = time_20_30 = max_vel_20_30 = pos_vel_20_30 = max_acc_20_30 = pos_acc_20_30 = mean_y_20_30 = variance_y_20_30 = vel_x_30_35= acc_x_30_35 = time_30_35 = max_vel_30_35= pos_vel_30_35= max_acc_30_35= pos_acc_30_35 = mean_y_30_35 = variance_y_30_35 = vel_x_30_40 = acc_x_30_40 = time_30_40 = max_vel_30_40 = pos_vel_30_40 = max_acc_30_40 = pos_acc_30_40 = mean_y_30_40 = variance_y_30_40 =vel_x_35_40 =acc_x_35_40 =time_35_40 =max_vel_35_40 =pos_vel_35_40 =max_acc_35_40 =pos_acc_35_40= mean_y_35_40 =variance_y_35_40 = None
        player_id = play_df.playerId.iloc[0]
        if player_id == 0.0:
            print(f"yards {subtraction} and val {player_id}")
        play_id = play 
        game_key = play_df.gameKey.iloc[0] 
        route = play_df.route.iloc[0] 
        metrics = [player_id,play_id,game_key,route,vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5,vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5,vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10,vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10,vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15,vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20,vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20,vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25,vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30,vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30,vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35,vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40,vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40]

    if subtraction >= 20:
        #buckets["20"].append(key)
        vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5 = metric_creation(play_df, 0, 2.5)
        vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5 = metric_creation(play_df,0,5)
        vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10 = metric_creation(play_df,5,10)
        vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10 = metric_creation(play_df,0,10)
        vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15 = metric_creation(play_df,10,15)
        vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20 = metric_creation(play_df,15,20)
        vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20 = metric_creation(play_df,10,20)
        vel_x_20_25 = acc_x_20_25 = time_20_25 = max_vel_20_25 = pos_vel_20_25 = max_acc_20_25 = pos_acc_20_25 = mean_y_20_25 = variance_y_20_25 = vel_x_25_30 = acc_x_25_30 = time_25_30 = max_vel_25_30 = pos_vel_25_30 = max_acc_25_30 = pos_acc_25_30 = mean_y_25_30 = variance_y_25_30 = vel_x_20_30 = acc_x_20_30 = time_20_30 = max_vel_20_30 = pos_vel_20_30 = max_acc_20_30 = pos_acc_20_30 = mean_y_20_30 = variance_y_20_30 = vel_x_30_35= acc_x_30_35 = time_30_35 = max_vel_30_35= pos_vel_30_35= max_acc_30_35= pos_acc_30_35 = mean_y_30_35 = variance_y_30_35 = vel_x_30_40 = acc_x_30_40 = time_30_40 = max_vel_30_40 = pos_vel_30_40 = max_acc_30_40 = pos_acc_30_40 = mean_y_30_40 = variance_y_30_40 =vel_x_35_40 =acc_x_35_40 =time_35_40 =max_vel_35_40 =pos_vel_35_40 =max_acc_35_40 =pos_acc_35_40= mean_y_35_40 =variance_y_35_40 = None
        player_id = play_df.playerId.iloc[0]
        if player_id == 0.0:
            print(f"yards {subtraction} and val {player_id}")
        play_id = play 
        game_key = play_df.gameKey.iloc[0] 
        route = play_df.route.iloc[0] 
        metrics = [player_id,play_id,game_key,route,vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5,vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5,vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10,vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10,vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15,vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20,vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20,vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25,vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30,vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30,vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35,vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40,vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40]

    if subtraction >= 25:
        #buckets["25"].append(key)
        vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5 = metric_creation(play_df, 0, 2.5)
        vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5 = metric_creation(play_df,0,5)
        vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10 = metric_creation(play_df,5,10)
        vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10 = metric_creation(play_df,0,10)
        vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15 = metric_creation(play_df,10,15)
        vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20 = metric_creation(play_df,15,20)
        vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20 = metric_creation(play_df,10,20)
        vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25 = metric_creation(play_df,20,25)
        vel_x_25_30 = acc_x_25_30 = time_25_30 = max_vel_25_30 = pos_vel_25_30 = max_acc_25_30 = pos_acc_25_30 = mean_y_25_30 = variance_y_25_30 = vel_x_20_30 = acc_x_20_30 = time_20_30 = max_vel_20_30 = pos_vel_20_30 = max_acc_20_30 = pos_acc_20_30 = mean_y_20_30 = variance_y_20_30 = vel_x_30_35= acc_x_30_35 = time_30_35 = max_vel_30_35= pos_vel_30_35= max_acc_30_35= pos_acc_30_35 = mean_y_30_35 = variance_y_30_35 = vel_x_30_40 = acc_x_30_40 = time_30_40 = max_vel_30_40 = pos_vel_30_40 = max_acc_30_40 = pos_acc_30_40 = mean_y_30_40 = variance_y_30_40 =vel_x_35_40 =acc_x_35_40 =time_35_40 =max_vel_35_40 =pos_vel_35_40 =max_acc_35_40 =pos_acc_35_40= mean_y_35_40 =variance_y_35_40 = None
        player_id = play_df.playerId.iloc[0]
        if player_id == 0.0:
            print(f"yards {subtraction} and val {player_id}")
        play_id = play 
        game_key = play_df.gameKey.iloc[0] 
        route = play_df.route.iloc[0] 
        metrics = [player_id,play_id,game_key,route,vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5,vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5,vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10,vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10,vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15,vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20,vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20,vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25,vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30,vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30,vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35,vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40,vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40]


    if subtraction >= 30:
        #buckets["30"].append(key)
        vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5 = metric_creation(play_df, 0, 2.5)
        vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5 = metric_creation(play_df,0,5)
        vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10 = metric_creation(play_df,5,10)
        vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10 = metric_creation(play_df,0,10)
        vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15 = metric_creation(play_df,10,15)
        vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20 = metric_creation(play_df,15,20)
        vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20 = metric_creation(play_df,10,20)
        vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25 = metric_creation(play_df,20,25)
        vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30 = metric_creation(play_df,25,30)
        vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30 = metric_creation(play_df,20,30)
        vel_x_30_35= acc_x_30_35 = time_30_35 = max_vel_30_35= pos_vel_30_35= max_acc_30_35= pos_acc_30_35 = mean_y_30_35 = variance_y_30_35 = vel_x_30_40 = acc_x_30_40 = time_30_40 = max_vel_30_40 = pos_vel_30_40 = max_acc_30_40 = pos_acc_30_40 = mean_y_30_40 = variance_y_30_40 =vel_x_35_40 =acc_x_35_40 =time_35_40 =max_vel_35_40 =pos_vel_35_40 =max_acc_35_40 =pos_acc_35_40= mean_y_35_40 =variance_y_35_40 = None
        player_id = play_df.playerId.iloc[0]
                  
        if player_id == 0.0:
            print(f"yards {subtraction} and val {player_id}")
        play_id = play 
        game_key = play_df.gameKey.iloc[0] 
        route = play_df.route.iloc[0] 
        metrics = [player_id,play_id,game_key,route,vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5,vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5,vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10,vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10,vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15,vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20,vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20,vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25,vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30,vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30,vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35,vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40,vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40]

    if subtraction >= 35:
        #buckets["35"].append(key)
        vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5 = metric_creation(play_df, 0, 2.5)
        vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5 = metric_creation(play_df,0,5)
        vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10 = metric_creation(play_df,5,10)
        vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10 = metric_creation(play_df,0,10)
        vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15 = metric_creation(play_df,10,15)
        vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20 = metric_creation(play_df,15,20)
        vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20 = metric_creation(play_df,10,20)
        vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25 = metric_creation(play_df,20,25)
        vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30 = metric_creation(play_df,25,30)
        vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30 = metric_creation(play_df,20,30)
        vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35 = metric_creation(play_df,30,35)
        vel_x_30_40 = acc_x_30_40 = time_30_40 = max_vel_30_40 = pos_vel_30_40 = max_acc_30_40 = pos_acc_30_40 = mean_y_30_40 = variance_y_30_40 =vel_x_35_40 =acc_x_35_40 =time_35_40 =max_vel_35_40 =pos_vel_35_40 =max_acc_35_40 =pos_acc_35_40= mean_y_35_40 =variance_y_35_40 = None
        player_id = play_df.playerId.iloc[0]
        if player_id == 0.0:
            print(f"yards {subtraction} and val {player_id}")
        play_id = play 
        game_key = play_df.gameKey.iloc[0] 
        route = play_df.route.iloc[0] 
        metrics = [player_id,play_id,game_key,route,vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5,vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5,vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10,vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10,vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15,vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20,vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20,vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25,vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30,vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30,vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35,vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40,vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40]

    if subtraction >= 40:
        #buckets["40"].append(key)
        vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5 = metric_creation(play_df, 0, 2.5)
        vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5 = metric_creation(play_df,0,5)
        vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10 = metric_creation(play_df,5,10)
        vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10 = metric_creation(play_df,0,10)
        vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15 = metric_creation(play_df,10,15)
        vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20 = metric_creation(play_df,15,20)
        vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20 = metric_creation(play_df,10,20)
        vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25 = metric_creation(play_df,20,25)
        vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30 = metric_creation(play_df,25,30)
        vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30 = metric_creation(play_df,20,30)
        vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35 = metric_creation(play_df,30,35)
        vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40 = metric_creation(play_df,30,40)
        vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40 = metric_creation(play_df,35,40)
        player_id = play_df.playerId.iloc[0]
        if player_id == 0.0:
            print(f"yards {subtraction} and val {player_id}")
        play_id = play 
        game_key = play_df.gameKey.iloc[0] 
        route = play_df.route.iloc[0] 
        metrics = [player_id,play_id,game_key,route,vel_x_2_5, acc_x_2_5, time_2_5, max_vel_2_5, pos_vel_2_5, max_acc_2_5, pos_acc_2_5, mean_y_2_5, variance_y_2_5,vel_x_5, acc_x_5, time_5, max_vel_5, pos_vel_5, max_acc_5, pos_acc_5, mean_y_5, variance_y_5,vel_x_5_10, acc_x_5_10, time_5_10, max_vel_5_10, pos_vel_5_10, max_acc_5_10, pos_acc_5_10, mean_y_5_10, variance_y_5_10,vel_x_10, acc_x_10, time_10, max_vel_10, pos_vel_10, max_acc_10, pos_acc_10, mean_y_10, variance_y_10,vel_x_10_15, acc_x_10_15, time_10_15, max_vel_10_15, pos_vel_10_15, max_acc_10_15, pos_acc_10_15, mean_y_10_15, variance_y_10_15,vel_x_15_20, acc_x_15_20, time_15_20, max_vel_15_20, pos_vel_15_20, max_acc_15_20, pos_acc_15_20, mean_y_15_20, variance_y_15_20,vel_x_10_20, acc_x_10_20, time_10_20, max_vel_10_20, pos_vel_10_20, max_acc_10_20, pos_acc_10_20, mean_y_10_20, variance_y_10_20,vel_x_20_25, acc_x_20_25, time_20_25, max_vel_20_25, pos_vel_20_25, max_acc_20_25, pos_acc_20_25, mean_y_20_25, variance_y_20_25,vel_x_25_30, acc_x_25_30, time_25_30, max_vel_25_30, pos_vel_25_30, max_acc_25_30, pos_acc_25_30, mean_y_25_30, variance_y_25_30,vel_x_20_30, acc_x_20_30, time_20_30, max_vel_20_30, pos_vel_20_30, max_acc_20_30, pos_acc_20_30, mean_y_20_30, variance_y_20_30,vel_x_30_35, acc_x_30_35, time_30_35, max_vel_30_35, pos_vel_30_35, max_acc_30_35, pos_acc_30_35, mean_y_30_35, variance_y_30_35,vel_x_30_40, acc_x_30_40, time_30_40, max_vel_30_40, pos_vel_30_40, max_acc_30_40, pos_acc_30_40, mean_y_30_40, variance_y_30_40,vel_x_35_40, acc_x_35_40, time_35_40, max_vel_35_40, pos_vel_35_40, max_acc_35_40, pos_acc_35_40, mean_y_35_40, variance_y_35_40]
        #print(len(metrics))
    metrics_Go.loc[len(metrics_Go)] = metrics
  return metrics_Go

In [6]:
columns = ['player_id','route','vel_x_2_5', 'acc_x_2_5', 'time_2_5', 'max_vel_2_5', 'pos_vel_2_5', 'max_acc_2_5', 'pos_acc_2_5', 'mean_y_2_5', 'variance_y_2_5','vel_x_5', 'acc_x_5', 'time_5', 'max_vel_5', 'pos_vel_5', 'max_acc_5', 'pos_acc_5', 'mean_y_5', 'variance_y_5','vel_x_5_10', 'acc_x_5_10', 'time_5_10', 'max_vel_5_10', 'pos_vel_5_10', 'max_acc_5_10', 'pos_acc_5_10', 'mean_y_5_10', 'variance_y_5_10','vel_x_10', 'acc_x_10', 'time_10', 'max_vel_10', 'pos_vel_10', 'max_acc_10', 'pos_acc_10', 'mean_y_10', 'variance_y_10','vel_x_10_15', 'acc_x_10_15', 'time_10_15', 'max_vel_10_15', 'pos_vel_10_15', 'max_acc_10_15', 'pos_acc_10_15', 'mean_y_10_15', 'variance_y_10_15','vel_x_15_20', 'acc_x_15_20', 'time_15_20', 'max_vel_15_20', 'pos_vel_15_20', 'max_acc_15_20', 'pos_acc_15_20', 'mean_y_15_20', 'variance_y_15_20','vel_x_10_20', 'acc_x_10_20', 'time_10_20', 'max_vel_10_20', 'pos_vel_10_20', 'max_acc_10_20', 'pos_acc_10_20', 'mean_y_10_20', 'variance_y_10_20','vel_x_20_25', 'acc_x_20_25', 'time_20_25', 'max_vel_20_25', 'pos_vel_20_25', 'max_acc_20_25', 'pos_acc_20_25', 'mean_y_20_25', 'variance_y_20_25','vel_x_25_30', 'acc_x_25_30', 'time_25_30', 'max_vel_25_30', 'pos_vel_25_30', 'max_acc_25_30', 'pos_acc_25_30', 'mean_y_25_30', 'variance_y_25_30','vel_x_20_30', 'acc_x_20_30', 'time_20_30', 'max_vel_20_30', 'pos_vel_20_30', 'max_acc_20_30', 'pos_acc_20_30', 'mean_y_20_30', 'variance_y_20_30','vel_x_30_35', 'acc_x_30_35', 'time_30_35', 'max_vel_30_35', 'pos_vel_30_35', 'max_acc_30_35', 'pos_acc_30_35', 'mean_y_30_35', 'variance_y_30_35','vel_x_30_40', 'acc_x_30_40', 'time_30_40', 'max_vel_30_40', 'pos_vel_30_40', 'max_acc_30_40', 'pos_acc_30_40', 'mean_y_30_40', 'variance_y_30_40','vel_x_35_40', 'acc_x_35_40', 'time_35_40', 'max_vel_35_40', 'pos_vel_35_40', 'max_acc_35_40', 'pos_acc_35_40', 'mean_y_35_40', 'variance_y_35_40']
go_route_metrics_df = pd.DataFrame(columns)
players = go_df.playerId.drop_duplicates().tolist()
for player in players:
    curr_metrics_df = pd.DataFrame(columns = ['player_id','play_id','game_key','route','vel_x_2_5', 'acc_x_2_5', 'time_2_5', 'max_vel_2_5', 'pos_vel_2_5', 'max_acc_2_5', 'pos_acc_2_5', 'mean_y_2_5', 'variance_y_2_5','vel_x_5', 'acc_x_5', 'time_5', 'max_vel_5', 'pos_vel_5', 'max_acc_5', 'pos_acc_5', 'mean_y_5', 'variance_y_5','vel_x_5_10', 'acc_x_5_10', 'time_5_10', 'max_vel_5_10', 'pos_vel_5_10', 'max_acc_5_10', 'pos_acc_5_10', 'mean_y_5_10', 'variance_y_5_10','vel_x_10', 'acc_x_10', 'time_10', 'max_vel_10', 'pos_vel_10', 'max_acc_10', 'pos_acc_10', 'mean_y_10', 'variance_y_10','vel_x_10_15', 'acc_x_10_15', 'time_10_15', 'max_vel_10_15', 'pos_vel_10_15', 'max_acc_10_15', 'pos_acc_10_15', 'mean_y_10_15', 'variance_y_10_15','vel_x_15_20', 'acc_x_15_20', 'time_15_20', 'max_vel_15_20', 'pos_vel_15_20', 'max_acc_15_20', 'pos_acc_15_20', 'mean_y_15_20', 'variance_y_15_20','vel_x_10_20', 'acc_x_10_20', 'time_10_20', 'max_vel_10_20', 'pos_vel_10_20', 'max_acc_10_20', 'pos_acc_10_20', 'mean_y_10_20', 'variance_y_10_20','vel_x_20_25', 'acc_x_20_25', 'time_20_25', 'max_vel_20_25', 'pos_vel_20_25', 'max_acc_20_25', 'pos_acc_20_25', 'mean_y_20_25', 'variance_y_20_25','vel_x_25_30', 'acc_x_25_30', 'time_25_30', 'max_vel_25_30', 'pos_vel_25_30', 'max_acc_25_30', 'pos_acc_25_30', 'mean_y_25_30', 'variance_y_25_30','vel_x_20_30', 'acc_x_20_30', 'time_20_30', 'max_vel_20_30', 'pos_vel_20_30', 'max_acc_20_30', 'pos_acc_20_30', 'mean_y_20_30', 'variance_y_20_30','vel_x_30_35', 'acc_x_30_35', 'time_30_35', 'max_vel_30_35', 'pos_vel_30_35', 'max_acc_30_35', 'pos_acc_30_35', 'mean_y_30_35', 'variance_y_30_35','vel_x_30_40', 'acc_x_30_40', 'time_30_40', 'max_vel_30_40', 'pos_vel_30_40', 'max_acc_30_40', 'pos_acc_30_40', 'mean_y_30_40', 'variance_y_30_40','vel_x_35_40', 'acc_x_35_40', 'time_35_40', 'max_vel_35_40', 'pos_vel_35_40', 'max_acc_35_40', 'pos_acc_35_40', 'mean_y_35_40', 'variance_y_35_40'])
    curr_df = go_df.loc[go_df.playerId == player]
    curr_metrics_df = curr_metrics_df.append(go_metric_creation(curr_df))
    curr_metrics_df = curr_metrics_df.fillna(value=np.nan)
    #print(curr_metrics_df)
    go_route_metrics_df = go_route_metrics_df.append(percentile_metrics(curr_metrics_df, columns))

# path to file location in github 
#path = 'in_game/data/go_route_metrics.csv'
#go_route_metrics_df.to_csv('go_route_metrics.csv')

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_metrics_df = curr_metrics_df.append(go_metric_creation(curr_df))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  go_route_metrics_df = go_route_metrics_df.append(percentile_metrics(curr_metrics_df, columns))


vel_x_2_5
acc_x_2_5
max_vel_2_5
pos_vel_2_5
max_acc_2_5
pos_acc_2_5
mean_y_2_5
variance_y_2_5
vel_x_5
acc_x_5
max_vel_5
pos_vel_5
max_acc_5
pos_acc_5
mean_y_5
variance_y_5
vel_x_5_10
acc_x_5_10
max_vel_5_10
pos_vel_5_10
max_acc_5_10
pos_acc_5_10
mean_y_5_10
variance_y_5_10
vel_x_10
acc_x_10
max_vel_10
pos_vel_10
max_acc_10
pos_acc_10
mean_y_10
variance_y_10
vel_x_10_15
acc_x_10_15
max_vel_10_15
pos_vel_10_15
max_acc_10_15
pos_acc_10_15
mean_y_10_15
variance_y_10_15
vel_x_15_20
acc_x_15_20
max_vel_15_20
pos_vel_15_20
max_acc_15_20
pos_acc_15_20
mean_y_15_20
variance_y_15_20
vel_x_10_20
acc_x_10_20
max_vel_10_20
pos_vel_10_20
max_acc_10_20
pos_acc_10_20
mean_y_10_20
variance_y_10_20
vel_x_20_25
acc_x_20_25
max_vel_20_25
pos_vel_20_25
max_acc_20_25
pos_acc_20_25
mean_y_20_25
variance_y_20_25
vel_x_25_30
acc_x_25_30
max_vel_25_30
pos_vel_25_30
max_acc_25_30
pos_acc_25_30
mean_y_25_30
variance_y_25_30
vel_x_20_30
acc_x_20_30
max_vel_20_30
pos_vel_20_30
max_acc_20_30
pos_acc_20_30
mean_y_20_

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_metrics_df = curr_metrics_df.append(go_metric_creation(curr_df))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  go_route_metrics_df = go_route_metrics_df.append(percentile_metrics(curr_metrics_df, columns))


vel_x_2_5
acc_x_2_5
max_vel_2_5
pos_vel_2_5
max_acc_2_5
pos_acc_2_5
mean_y_2_5
variance_y_2_5
vel_x_5
acc_x_5
max_vel_5
pos_vel_5
max_acc_5
pos_acc_5
mean_y_5
variance_y_5
vel_x_5_10
acc_x_5_10
max_vel_5_10
pos_vel_5_10
max_acc_5_10
pos_acc_5_10
mean_y_5_10
variance_y_5_10
vel_x_10
acc_x_10
max_vel_10
pos_vel_10
max_acc_10
pos_acc_10
mean_y_10
variance_y_10
vel_x_10_15
acc_x_10_15
max_vel_10_15
pos_vel_10_15
max_acc_10_15
pos_acc_10_15
mean_y_10_15
variance_y_10_15
vel_x_15_20
acc_x_15_20
max_vel_15_20
pos_vel_15_20
max_acc_15_20
pos_acc_15_20
mean_y_15_20
variance_y_15_20
vel_x_10_20
acc_x_10_20
max_vel_10_20
pos_vel_10_20
max_acc_10_20
pos_acc_10_20
mean_y_10_20
variance_y_10_20
vel_x_20_25
acc_x_20_25
max_vel_20_25
pos_vel_20_25
max_acc_20_25
pos_acc_20_25
mean_y_20_25
variance_y_20_25
vel_x_25_30
acc_x_25_30
max_vel_25_30
pos_vel_25_30
max_acc_25_30
pos_acc_25_30
mean_y_25_30
variance_y_25_30
vel_x_20_30
acc_x_20_30
max_vel_20_30
pos_vel_20_30
max_acc_20_30
pos_acc_20_30
mean_y_20_

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_metrics_df = curr_metrics_df.append(go_metric_creation(curr_df))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  go_route_metrics_df = go_route_metrics_df.append(percentile_metrics(curr_metrics_df, columns))


...

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  go_route_metrics_df = go_route_metrics_df.append(percentile_metrics(curr_metrics_df, columns))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_metrics_df = curr_metrics_df.append(go_metric_creation(curr_df))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  go_route_metrics_df = go_route_metrics_df.append(percentile_metrics(curr_metrics_df, columns))


vel_x_2_5
acc_x_2_5
max_vel_2_5
pos_vel_2_5
max_acc_2_5
pos_acc_2_5
mean_y_2_5
variance_y_2_5
vel_x_5
acc_x_5
max_vel_5
pos_vel_5
max_acc_5
pos_acc_5
mean_y_5
variance_y_5
vel_x_5_10
acc_x_5_10
max_vel_5_10
pos_vel_5_10
max_acc_5_10
pos_acc_5_10
mean_y_5_10
variance_y_5_10
vel_x_10
acc_x_10
max_vel_10
pos_vel_10
max_acc_10
pos_acc_10
mean_y_10
variance_y_10
vel_x_10_15
acc_x_10_15
max_vel_10_15
pos_vel_10_15
max_acc_10_15
pos_acc_10_15
mean_y_10_15
variance_y_10_15
vel_x_15_20
acc_x_15_20
max_vel_15_20
pos_vel_15_20
max_acc_15_20
pos_acc_15_20
mean_y_15_20
variance_y_15_20
vel_x_10_20
acc_x_10_20
max_vel_10_20
pos_vel_10_20
max_acc_10_20
pos_acc_10_20
mean_y_10_20
variance_y_10_20
vel_x_20_25
acc_x_20_25
max_vel_20_25
pos_vel_20_25
max_acc_20_25
pos_acc_20_25
mean_y_20_25
variance_y_20_25
vel_x_25_30
acc_x_25_30
max_vel_25_30
pos_vel_25_30
max_acc_25_30
pos_acc_25_30
mean_y_25_30
variance_y_25_30
vel_x_20_30
acc_x_20_30
max_vel_20_30
pos_vel_20_30
max_acc_20_30
pos_acc_20_30
mean_y_20_

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_metrics_df = curr_metrics_df.append(go_metric_creation(curr_df))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  go_route_metrics_df = go_route_metrics_df.append(percentile_metrics(curr_metrics_df, columns))


vel_x_2_5
acc_x_2_5
max_vel_2_5
pos_vel_2_5
max_acc_2_5
pos_acc_2_5
mean_y_2_5
variance_y_2_5
vel_x_5
acc_x_5
max_vel_5
pos_vel_5
max_acc_5
pos_acc_5
mean_y_5
variance_y_5
vel_x_5_10
acc_x_5_10
max_vel_5_10
pos_vel_5_10
max_acc_5_10
pos_acc_5_10
mean_y_5_10
variance_y_5_10
vel_x_10
acc_x_10
max_vel_10
pos_vel_10
max_acc_10
pos_acc_10
mean_y_10
variance_y_10
vel_x_10_15
acc_x_10_15
max_vel_10_15
pos_vel_10_15
max_acc_10_15
pos_acc_10_15
mean_y_10_15
variance_y_10_15
vel_x_15_20
acc_x_15_20
max_vel_15_20
pos_vel_15_20
max_acc_15_20
pos_acc_15_20
mean_y_15_20
variance_y_15_20
vel_x_10_20
acc_x_10_20
max_vel_10_20
pos_vel_10_20
max_acc_10_20
pos_acc_10_20
mean_y_10_20
variance_y_10_20
vel_x_20_25
acc_x_20_25
max_vel_20_25
pos_vel_20_25
max_acc_20_25
pos_acc_20_25
mean_y_20_25
variance_y_20_25
vel_x_25_30
acc_x_25_30
max_vel_25_30
pos_vel_25_30
max_acc_25_30
pos_acc_25_30
mean_y_25_30
variance_y_25_30
vel_x_20_30
acc_x_20_30
max_vel_20_30
pos_vel_20_30
max_acc_20_30
pos_acc_20_30
mean_y_20_

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_metrics_df = curr_metrics_df.append(go_metric_creation(curr_df))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1678468272.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  go_route_metrics_df = go_route_metrics_df.append(percentile_metrics(curr_metrics_df, columns))


In [7]:
go_route_metrics_df = go_route_metrics_df.dropna(subset=['player_id'])

In [8]:
go_route_metrics_df_final = go_route_metrics_df.drop(0, axis=1)

In [9]:
go_route_metrics_df_final

,player_id,route,vel_x_2_5,acc_x_2_5,time_2_5,max_vel_2_5,pos_vel_2_5,max_acc_2_5,pos_acc_2_5,mean_y_2_5,...,variance_y_30_40,vel_x_35_40,acc_x_35_40,time_35_40,max_vel_35_40,pos_vel_35_40,max_acc_35_40,pos_acc_35_40,mean_y_35_40,variance_y_35_40
0,54476.0,GO,2.902841,5.913048,1.079310,5.569037,2.745952,9.824836,1.499926,0.599515,...,0.327907,13.737758,1.243891,0.440000,14.065728,0.914172,2.631188,4.223455,5.933769,0.067068
0,54517.0,GO,2.654072,5.306639,1.137069,5.464999,2.773788,10.114152,1.690002,1.113163,...,0.164485,0.164485,0.164485,0.000000,0.164485,0.164485,0.164485,0.164485,0.164485,0.164485
0,54475.0,GO,2.820113,5.668838,1.104545,5.756861,2.901639,10.699269,1.508413,2.125292,...,0.164485,0.164485,0.164485,0.000000,0.164485,0.164485,0.164485,0.164485,0.164485,0.164485
0,54481.0,GO,2.744812,5.418318,1.138462,5.999429,3.126546,9.698923,1.371723,1.015509,...,0.164485,0.164485,0.164485,0.000000,0.164485,0.164485,0.164485,0.164485,0.164485,0.164485
0,54519.0,GO,2.392600,4.787187,1.304348,5.034928,2.555158,10.207924,1.819552,1.474014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,54499.0,GO,2.593906,5.369461,1.141463,5.706787,2.825255,11.019592,1.348320,0.987822,...,1.385214,12.118288,0.928920,0.716667,12.510978,1.783400,5.762129,3.935541,6.372281,0.676372
0,54628.0,GO,2.160644,4.486781,1.266667,5.047475,2.451451,8.419586,1.569108,3.763658,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,54483.0,GO,2.452742,5.401115,1.134286,5.302241,2.493173,9.969007,1.503016,0.643179,...,0.435435,10.817334,0.276304,0.600000,10.602712,0.959186,2.713186,2.085081,7.309661,0.094403
0,54508.0,GO,2.523074,4.994662,1.162500,4.996778,2.386608,7.961680,1.152252,2.240791,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,54564.0,GO,2.398590,4.904388,1.181081,5.090846,2.530017,9.260694,1.124504,0.997808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
go_metrics_path = r'C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\analysis\data\correlation\go_metrics.csv'
go_route_metrics_df_final.to_csv(go_metrics_path)

# In and Out Metrics

In [12]:
def separate_segments_in_route(df,playId):
    move_array0 = []
    move_array1 = []
    move_array2 = []
    move_array3 = []
    total_time = 0
    straight = False
    turn1 = False
    end = False
    
    curr_player_df = df.loc[df.playId == playId]
    max_vel_x = curr_player_df.inverted_vel_x.max()
    right = curr_player_df.inverted_y.iloc[-1] - curr_player_df.inverted_y.iloc[-4]
    if right >= 0:
        direction = "left"
    else:
        direction = "right"
    curr_player_df['direction'] = direction
    if max_vel_x > 4:
        for i in range(len(curr_player_df)):
            curr_df = curr_player_df.iloc[i]
            move_array0.append(curr_df)
            if len(move_array1) == 0:
                start = curr_df
                straight = True
            if straight and turn1 == False and end == False and curr_df.inverted_acc_x <-0.1 and curr_df.inverted_vel_x >4:
                turn1 = True
            if straight and turn1 and end == False and curr_df.inverted_vel_x> 100:
                end = True 
            if straight and turn1 == False and end == False:
                move_array1.append(curr_df)
            if straight and turn1 and end == False and ((right > 0 and curr_df.inverted_vel_y >2) or (right < 0 and curr_df.inverted_vel_y <-2)):
                move_array2.append(curr_df)
            if straight and turn1 and end:
                move_array3.append(curr_df)
    elif max_vel_x > 2:
        for i in range(len(curr_player_df)):
            curr_df = curr_player_df.iloc[i]
            move_array0.append(curr_df)
            if len(move_array1) == 0:
                start = curr_df
                straight = True
            if straight and turn1 == False and end == False and curr_df.inverted_acc_x <-0.1 and curr_df.inverted_vel_x >2:
                turn1 = True
            if straight and turn1 and end == False and curr_df.inverted_vel_x>100:
                end = True
            if straight and turn1 == False and end == False:
                move_array1.append(curr_df)
            if straight and turn1 and end == False and ((right > 0 and curr_df.inverted_vel_y >1 ) or (right < 0 and curr_df.inverted_vel_y <-1)):
                move_array2.append(curr_df)
            if straight and turn1 and end:
                move_array3.append(curr_df)
    else: return pd.DataFrame()


    df1 = pd.DataFrame(move_array1, index =[i for i in range(len(move_array1))],columns =['gameKey', 'playId', 'playerId', 'ts', 'x', 'y', 's', 'a', 'event','route', 'acc_x', 'acc_y', 'left_or_right_route', 'inverted_x','inverted_y', 'inverted_vel_x', 'inverted_vel_y', 'inverted_acc_x','inverted_acc_y','direction'])
    df2 = pd.DataFrame(move_array2, index =[i for i in range(len(move_array2))],columns =['gameKey', 'playId', 'playerId', 'ts', 'x', 'y', 's', 'a', 'event','route', 'acc_x', 'acc_y', 'left_or_right_route', 'inverted_x','inverted_y', 'inverted_vel_x', 'inverted_vel_y', 'inverted_acc_x','inverted_acc_y','direction'])
    #df3 = pd.DataFrame(move_array3, index =[i for i in range(len(move_array3))],columns =['playerId', 'timeStamp', 'posX', 'posY', 'velX','velY', 'accX', 'accY', 'acc','vel','firstName','lastName','Group','position','totalTime'])
    df1['segment'] = 'first'
    df2['segment'] = 'second'
    #df3['segment'] = 'third'
    
    final = df1.append(df2)
    #final = final.append(df3)
    return final

In [13]:
players = in_df.playerId.drop_duplicates().tolist()
in_segmented = pd.DataFrame(columns =['gameKey', 'playId', 'playerId', 'ts', 'x', 'y', 's', 'a', 'event','route', 'acc_x', 'acc_y', 'left_or_right_route', 'inverted_x','inverted_y', 'inverted_vel_x', 'inverted_vel_y', 'inverted_acc_x','inverted_acc_y','direction','segment'])
for player in players:
    curr_df = in_df.loc[in_df.playerId == player]
    playIds = curr_df.playId.drop_duplicates().tolist()
    for play in playIds:
        in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))

# path to file location in github
path = r"C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\analysis\data\correlation\in_routes_segmented.csv"
in_segmented.to_csv(path)

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.ap

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.app

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.ap

...

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.ap

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.ap

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3492753896.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_segmented = in_segmented.append(separate_segments_in_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3833034892.py:63: FutureWarning: The frame.ap

In [14]:
in_segmented_route_left = in_segmented.loc[in_segmented.direction == 'left']
in_segmented_route_right = in_segmented.loc[in_segmented.direction == 'right']

### Metric Functions

In [15]:
def straight1_metrics_in_game(straight_df):
    pos_vel = straight_df.loc[straight_df.inverted_vel_x == straight_df.inverted_vel_x.max()]
    pos_vel_final = pos_vel.iloc[0]['inverted_x']
    pos_acc = straight_df.loc[straight_df.inverted_acc_x == straight_df.inverted_acc_x.max()]
    pos_acc_final = pos_acc.iloc[0]['inverted_x']
    return straight_df.inverted_vel_x.mean(), straight_df.inverted_acc_x.mean(), (len(straight_df)-1)/10, straight_df.inverted_vel_x.max(), pos_vel_final, straight_df.inverted_acc_x.max(), pos_acc_final, straight_df.inverted_y.mean(), straight_df.inverted_y.std()**2

In [16]:
def turn1_metrics_in_game(turn_df):
    #Average metrics
    vel_x_avg_t1 = turn_df.inverted_vel_x.mean()
    vel_y_avg_t1 = turn_df.inverted_vel_y.mean()
    vel_avg_t1 = turn_df.s.mean()
    acc_x_avg_t1 = turn_df.inverted_acc_x.mean()
    acc_y_avg_t1 = turn_df.inverted_acc_y.mean()
    acc_avg_t1 = turn_df.a.mean()
    
    #Max/min metrics
    vel_x_max_t1 = turn_df.inverted_vel_x.max()
    vel_y_max_t1 = turn_df.inverted_vel_y.min()
    vel_max_t1 = turn_df.s.max()
    acc_x_min_t1 = turn_df.inverted_acc_x.min()
    acc_y_min_t1 = turn_df.inverted_acc_y.min()
    
    #Entering metrics
    vel_x_enter_t1 = turn_df.inverted_vel_x.iloc[0]
    vel_y_enter_t1 = turn_df.inverted_vel_y.iloc[0]
    vel_enter_t1 = turn_df.s.iloc[0]
    acc_x_enter_t1 = turn_df.inverted_acc_x.iloc[0]
    acc_y_enter_t1 = turn_df.inverted_acc_y.iloc[0]
    acc_enter_t1 = turn_df.a.iloc[0]
    
    #Leaving metrics
    vel_x_end_t1 = turn_df.inverted_vel_x.iloc[len(turn_df)-1]
    vel_y_end_t1 = turn_df.inverted_vel_y.iloc[len(turn_df)-1]
    vel_end_t1 = turn_df.s.iloc[len(turn_df)-1]
    acc_x_end_t1 = turn_df.inverted_acc_x.iloc[len(turn_df)-1]
    acc_y_end_t1 = turn_df.inverted_acc_y.iloc[len(turn_df)-1]
    acc_end_t1 = turn_df.a.iloc[len(turn_df)-1]
    
    #Displacement X and Y:
    change_x_t1 = math.fabs(turn_df.inverted_x.iloc[len(turn_df)-1] - turn_df.inverted_x.iloc[0])
    change_y_t1 = math.fabs(turn_df.inverted_y.iloc[len(turn_df)-1] - turn_df.inverted_y.iloc[0])
    change_pos_t1 = pow(pow(change_x_t1,2) + pow(change_y_t1,2),0.5)
   
    #Angle of turn
    #center = turn_df.iloc[math.trunc((len(turn_df)-1)/2)]
    #angle_t1 = getAngle((turn_df.posX.iloc[0], turn_df.posY.iloc[0]), (center.posX, center.posY), (turn_df.posX.iloc[len(turn_df)-1], turn_df.posY.iloc[len(turn_df)-1]))
    
    
    return vel_x_avg_t1, vel_y_avg_t1, vel_avg_t1, acc_x_avg_t1,acc_y_avg_t1,acc_avg_t1, vel_x_enter_t1, vel_y_enter_t1, vel_enter_t1, acc_x_enter_t1, acc_y_enter_t1, acc_enter_t1, vel_x_end_t1, vel_y_end_t1,vel_end_t1,acc_x_end_t1, acc_y_end_t1,acc_end_t1, change_x_t1, change_y_t1, change_pos_t1 #angle_t1

In [17]:
def metric_combiner(df,play):
    df = df.loc[df.playId == play] 
    route = df.route[0]
    straight_1 = df.loc[df.segment == 'first']
    vel_x_s1, acc_x_s1, time_s1, max_vel_x_s1, pos_vel_x_s1, max_acc_x_s1, pos_acc_x_s1, mean_y_s1, variance_y_s1 = straight1_metrics_in_game(straight_1)
    
    if len(df.segment.drop_duplicates().tolist()) >1:
        turn_1 = df.loc[df.segment == 'second']
        vel_x_avg_t1, vel_y_avg_t1, vel_avg_t1, acc_x_avg_t1,acc_y_avg_t1,acc_avg_t1, vel_x_enter_t1, vel_y_enter_t1, vel_enter_t1, acc_x_enter_t1, acc_y_enter_t1, acc_enter_t1, vel_x_end_t1, vel_y_end_t1,vel_end_t1,acc_x_end_t1, acc_y_end_t1,acc_end_t1, change_x_t1, change_y_t1, change_pos_t1 = turn1_metrics_in_game(turn_1)
    else:
        vel_x_avg_t1, vel_y_avg_t1, vel_avg_t1, acc_x_avg_t1,acc_y_avg_t1,acc_avg_t1, vel_x_enter_t1, vel_y_enter_t1, vel_enter_t1, acc_x_enter_t1, acc_y_enter_t1, acc_enter_t1, vel_x_end_t1, vel_y_end_t1,vel_end_t1,acc_x_end_t1, acc_y_end_t1,acc_end_t1, change_x_t1, change_y_t1, change_pos_t1 = None, None, None, None,None,None, None, None, None, None, None, None, None, None,None,None, None,None, None, None, None
    combined_metrics = [player,route,vel_x_s1, acc_x_s1, time_s1, max_vel_x_s1, pos_vel_x_s1, max_acc_x_s1, pos_acc_x_s1, mean_y_s1, variance_y_s1, vel_x_avg_t1, vel_y_avg_t1, vel_avg_t1, acc_x_avg_t1,acc_y_avg_t1,acc_avg_t1, vel_x_enter_t1, vel_y_enter_t1, vel_enter_t1, acc_x_enter_t1, acc_y_enter_t1, acc_enter_t1, vel_x_end_t1, vel_y_end_t1,vel_end_t1,acc_x_end_t1, acc_y_end_t1,acc_end_t1, change_x_t1, change_y_t1, change_pos_t1]
    return combined_metrics

### In route metrics

In [18]:
players = in_segmented_route_right['playerId'].drop_duplicates().tolist()
columns  = ['player_id', 'route','vel_x_straight', 'acc_x_straight', 'time_straight', 'max_vel_x_straight', 'pos_vel_x_straight', 'max_acc_x_straight', 'pos_acc_x_straight', 'mean_y_straight', 'variance_y_straight','vel_x_avg_turn_right', 'vel_y_avg_turn_right', 'vel_avg_turn_right', 'acc_x_avg_turn_right','acc_y_avg_turn_right','acc_avg_turn_right','vel_x_enter_turn_right', 'vel_y_enter_turn_right', 'vel_enter_turn_right', 'acc_x_enter_turn_right', 'acc_y_enter_turn_right', 'acc_enter_turn_right', 'vel_x_end_turn_right', 'vel_y_end_turn_right','vel_end_turn_right','acc_x_end_turn_right', 'acc_y_end_turn_right','acc_end_turn_right', 'change_x_turn_right', 'change_y_turn_right', 'change_pos_turn_right']
in_route_metrics_right = pd.DataFrame(columns)
for player in players:
    curr_df = pd.DataFrame(columns = columns)
    player_df = in_segmented_route_right.loc[in_segmented_route_right.playerId == player]
    playIds = player_df['playId'].drop_duplicates().tolist()
    for play in playIds:
        curr_df.loc[len(curr_df)] = metric_combiner(player_df,play)
    curr_df['route'] = 'in'
    in_route_metrics_right = in_route_metrics_right.append(percentile_metrics(curr_df,columns ))

columns = ['player_id', 'route','vel_x_straight', 'acc_x_straight', 'time_straight', 'max_vel_x_straight', 'pos_vel_x_straight', 'max_acc_x_straight', 'pos_acc_x_straight', 'mean_y_straight', 'variance_y_straight','vel_x_avg_turn_left', 'vel_y_avg_turn_left', 'vel_avg_turn_left', 'acc_x_avg_turn_left','acc_y_avg_turn_left','acc_avg_turn_left','vel_x_enter_turn_left', 'vel_y_enter_turn_left', 'vel_enter_turn_left', 'acc_x_enter_turn_left', 'acc_y_enter_turn_left', 'acc_enter_turn_left', 'vel_x_end_turn_left', 'vel_y_end_turn_left','vel_end_turn_left','acc_x_end_turn_left', 'acc_y_end_turn_left','acc_end_turn_left', 'change_x_turn_left', 'change_y_turn_left', 'change_pos_turn_left']
players = in_segmented_route_left['playerId'].drop_duplicates().tolist()
in_route_metrics_left = pd.DataFrame(columns = columns)
for player in players:
    curr_df = pd.DataFrame(columns = columns)
    player_df = in_segmented_route_left.loc[in_segmented_route_left.playerId == player]
    playIds = player_df['playId'].drop_duplicates().tolist()
    for play in playIds:
        curr_df.loc[len(curr_df)] = metric_combiner(player_df,play)
    curr_df['route'] = 'in'
    in_route_metrics_left = in_route_metrics_left.append(percentile_metrics(curr_df,columns))
    

# paths to the file location in the github
#path_right = 'in_game/data/in_route_metrics_right.csv'
#path_left = 'in_game/data/in_route_metrics_left.csv'
path_left = r"C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\analysis\data\correlation\in_route_metrics_left.csv"
path_right = r"C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\analysis\data\correlation\in_route_metrics_right.csv"
in_route_metrics_right.to_csv(path_right)
in_route_metrics_left.to_csv(path_left)

vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_right
vel_y_avg_turn_right
vel_avg_turn_right
acc_x_avg_turn_right
acc_y_avg_turn_right
acc_avg_turn_right
vel_x_enter_turn_right
vel_y_enter_turn_right
vel_enter_turn_right
acc_x_enter_turn_right
acc_y_enter_turn_right
acc_enter_turn_right
vel_x_end_turn_right
vel_y_end_turn_right
vel_end_turn_right
acc_x_end_turn_right
acc_y_end_turn_right
acc_end_turn_right
change_x_turn_right
change_y_turn_right
change_pos_turn_right


C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_right = in_route_metrics_right.append(percentile_metrics(curr_df,columns ))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_right = in_route_metrics_right.append(percentile_metrics(curr_df,columns ))


vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_right
vel_y_avg_turn_right
vel_avg_turn_right
acc_x_avg_turn_right
acc_y_avg_turn_right
acc_avg_turn_right
vel_x_enter_turn_right
vel_y_enter_turn_right
vel_enter_turn_right
acc_x_enter_turn_right
acc_y_enter_turn_right
acc_enter_turn_right
vel_x_end_turn_right
vel_y_end_turn_right
vel_end_turn_right
acc_x_end_turn_right
acc_y_end_turn_right
acc_end_turn_right
change_x_turn_right
change_y_turn_right
change_pos_turn_right
vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_right
vel_y_avg_turn_right
vel_avg_turn_right
acc_x_avg_turn_right
acc_y_avg_turn_right
acc_avg_turn_right
vel_x_enter_turn_right
vel_y_enter_turn_right
vel_enter_turn_right
acc_x_enter_turn_right
acc_y_enter_turn_right
acc_enter_turn_right
vel_x_end_turn_right

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_right = in_route_metrics_right.append(percentile_metrics(curr_df,columns ))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_right = in_route_metrics_right.append(percentile_metrics(curr_df,columns ))


vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_right
vel_y_avg_turn_right
vel_avg_turn_right
acc_x_avg_turn_right
acc_y_avg_turn_right
acc_avg_turn_right
vel_x_enter_turn_right
vel_y_enter_turn_right
vel_enter_turn_right
acc_x_enter_turn_right
acc_y_enter_turn_right
acc_enter_turn_right
vel_x_end_turn_right
vel_y_end_turn_right
vel_end_turn_right
acc_x_end_turn_right
acc_y_end_turn_right
acc_end_turn_right
change_x_turn_right
change_y_turn_right
change_pos_turn_right
vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_right
vel_y_avg_turn_right
vel_avg_turn_right
acc_x_avg_turn_right
acc_y_avg_turn_right
acc_avg_turn_right
vel_x_enter_turn_right
vel_y_enter_turn_right
vel_enter_turn_right
acc_x_enter_turn_right
acc_y_enter_turn_right
acc_enter_turn_right
vel_x_end_turn_right

...

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_left = in_route_metrics_left.append(percentile_metrics(curr_df,columns))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_left = in_route_metrics_left.append(percentile_metrics(curr_df,columns))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_left = in_route_metrics_left.append(percentile_metrics(curr_df,columns))


vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_left
vel_y_avg_turn_left
vel_avg_turn_left
acc_x_avg_turn_left
acc_y_avg_turn_left
acc_avg_turn_left
vel_x_enter_turn_left
vel_y_enter_turn_left
vel_enter_turn_left
acc_x_enter_turn_left
acc_y_enter_turn_left
acc_enter_turn_left
vel_x_end_turn_left
vel_y_end_turn_left
vel_end_turn_left
acc_x_end_turn_left
acc_y_end_turn_left
acc_end_turn_left
change_x_turn_left
change_y_turn_left
change_pos_turn_left
vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_left
vel_y_avg_turn_left
vel_avg_turn_left
acc_x_avg_turn_left
acc_y_avg_turn_left
acc_avg_turn_left
vel_x_enter_turn_left
vel_y_enter_turn_left
vel_enter_turn_left
acc_x_enter_turn_left
acc_y_enter_turn_left
acc_enter_turn_left
vel_x_end_turn_left
vel_y_end_turn_left
vel_end_turn_

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_left = in_route_metrics_left.append(percentile_metrics(curr_df,columns))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_left = in_route_metrics_left.append(percentile_metrics(curr_df,columns))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_left = in_route_metrics_left.append(percentile_metrics(curr_df,columns))


vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_left
vel_y_avg_turn_left
vel_avg_turn_left
acc_x_avg_turn_left
acc_y_avg_turn_left
acc_avg_turn_left
vel_x_enter_turn_left
vel_y_enter_turn_left
vel_enter_turn_left
acc_x_enter_turn_left
acc_y_enter_turn_left
acc_enter_turn_left
vel_x_end_turn_left
vel_y_end_turn_left
vel_end_turn_left
acc_x_end_turn_left
acc_y_end_turn_left
acc_end_turn_left
change_x_turn_left
change_y_turn_left
change_pos_turn_left
vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_left
vel_y_avg_turn_left
vel_avg_turn_left
acc_x_avg_turn_left
acc_y_avg_turn_left
acc_avg_turn_left
vel_x_enter_turn_left
vel_y_enter_turn_left
vel_enter_turn_left
acc_x_enter_turn_left
acc_y_enter_turn_left
acc_enter_turn_left
vel_x_end_turn_left
vel_y_end_turn_left
vel_end_turn_

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1049551784.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_route_metrics_left = in_route_metrics_left.append(percentile_metrics(curr_df,columns))


In [19]:
in_route_metrics_left

,player_id,route,vel_x_straight,acc_x_straight,time_straight,max_vel_x_straight,pos_vel_x_straight,max_acc_x_straight,pos_acc_x_straight,mean_y_straight,...,acc_enter_turn_left,vel_x_end_turn_left,vel_y_end_turn_left,vel_end_turn_left,acc_x_end_turn_left,acc_y_end_turn_left,acc_end_turn_left,change_x_turn_left,change_y_turn_left,change_pos_turn_left
0,54476,in,4.672886,3.312911,1.800000,7.835601,10.596220,10.217450,2.377508,0.324236,...,6.023081,0.064485,3.261213,3.104765,0.322427,-2.032720,4.979434,8.874329,22.584023,24.264988
0,54564,in,3.470208,3.791122,1.500000,6.258118,7.236021,8.014526,0.571207,0.971981,...,7.461414,1.744533,7.035932,7.059487,2.384826,0.384826,2.124134,6.434880,15.697738,16.955675
0,54475,in,4.622842,4.098292,1.892857,8.208850,10.581651,10.188045,2.697479,2.464326,...,6.231952,1.853821,7.957090,7.964795,0.893053,1.294915,3.698496,6.937326,18.957316,19.879967
0,54481,in,5.206838,3.560334,1.957143,8.652117,12.229700,9.191357,1.665680,0.415347,...,6.673868,2.855986,6.352289,6.886198,2.527634,3.199589,3.976738,4.912372,11.729405,12.402441
0,54519,in,4.826420,3.523866,2.014286,8.215149,10.363461,11.364728,2.152581,3.026128,...,6.138625,3.856259,7.908975,8.264810,4.085149,5.662770,5.675690,7.855324,26.657569,27.663369
0,54499,in,4.691687,4.521310,2.333333,8.154315,10.858617,8.343017,1.695428,1.311756,...,5.809625,4.237833,8.976619,8.888773,0.164485,-2.677573,4.911582,6.118024,13.010321,12.683851
0,54508,in,2.750251,3.476833,1.350000,4.790184,3.203456,7.322427,1.221166,-0.229279,...,6.853736,1.593456,4.283640,4.305276,1.967280,-0.065439,3.917614,1.679905,8.285931,8.294881
0,54628,in,5.233997,3.099839,2.133333,8.366717,11.832340,7.108725,1.205173,0.105743,...,6.809030,1.913956,3.315448,4.099844,-1.281840,0.536030,5.053737,6.484728,19.883327,20.897572
0,54517,in,5.180365,4.611815,2.130000,8.236181,11.568489,9.480666,1.726098,0.905541,...,5.141753,3.222741,4.844058,5.630371,0.803602,2.658276,4.723232,5.405017,19.577872,19.991014
0,54483,in,3.709578,4.531119,2.325000,6.747387,6.498175,10.408590,1.618958,2.272066,...,7.413497,1.276516,5.152103,5.185378,2.046520,-0.557942,3.453595,7.053843,16.538103,17.645130


### Out in game metrics

In [22]:
def separate_segments_out_route(df,playId):
    #df['acc'] = pow(pow(df.accX,2) + pow(df.accY,2),0.5)
    #df['vel'] = pow(pow(df.velX,2) + pow(df.velY,2),0.5)
    move_array0 = []
    move_array1 = []
    move_array2 = []
    move_array3 = []
    total_time = 0
    straight = False
    turn1 = False
    end = False
    
    curr_player_df = df.loc[df.playId == playId]
    max_vel_x = curr_player_df.inverted_vel_x.max()
    right = curr_player_df.inverted_y.iloc[-1] - curr_player_df.inverted_y.iloc[-4]
    #print(right)
    if right >= 0:
        direction = "left"
    else:
        direction = "right"
    curr_player_df['direction'] = direction
    if max_vel_x > 4:
        for i in range(len(curr_player_df)):
            curr_df = curr_player_df.iloc[i]
            move_array0.append(curr_df)
            #print(f"velocity x {curr_df.inverted_vel_x} and velocity y {curr_df.inverted_vel_y} and Direction:{curr_df.left_or_right_route}")
            #print(f"acceleration x {curr_df.inverted_acc_x} and acceleration y {curr_df.inverted_acc_y}")
            if len(move_array1) == 0:
                start = curr_df
                straight = True
            if straight and turn1 == False and end == False and curr_df.inverted_acc_x <-0.1 and curr_df.inverted_vel_x >4:
                turn1 = True
            if straight and turn1 and end == False and curr_df.inverted_vel_x> 100:
                end = True 
            if straight and turn1 == False and end == False:
                move_array1.append(curr_df)
                #print(f"acceleration x {curr_df.inverted_acc_x} and acceleration y {curr_df.inverted_acc_y}")
            if straight and turn1 and end == False and ((right > 0 and curr_df.inverted_vel_y >2) or (right < 0 and curr_df.inverted_vel_y <-2)):
                move_array2.append(curr_df)
                #print(f"speed {curr_df.s}")
            if straight and turn1 and end:
                #move_array3.append(curr_df)
                print("wrong")
    elif max_vel_x > 2:
        for i in range(len(curr_player_df)):
            curr_df = curr_player_df.iloc[i]
            move_array0.append(curr_df)
            #print(f"velocity x {curr_df.inverted_vel_x} and velocity y {curr_df.inverted_vel_y} and Direction:{curr_df.left_or_right_route}")
            #print(f"acceleration x {curr_df.inverted_acc_x} and acceleration y {curr_df.inverted_acc_y}")
            if len(move_array1) == 0:
                start = curr_df
                straight = True
            if straight and turn1 == False and end == False and curr_df.inverted_acc_x <-0.1 and curr_df.inverted_vel_x >2:
                turn1 = True
            if straight and turn1 and end == False and curr_df.inverted_vel_x>100:
                end = True
            if straight and turn1 == False and end == False:
                move_array1.append(curr_df)
                #print(f"acceleration x {curr_df.inverted_acc_x} and acceleration y {curr_df.inverted_acc_y}")
            if straight and turn1 and end == False and ((right > 0 and curr_df.inverted_vel_y >1 ) or (right < 0 and curr_df.inverted_vel_y <-1)):
                move_array2.append(curr_df)
                #print(move_array2)
                #print("test")
            if straight and turn1 and end:
                move_array3.append(curr_df)
    else: return pd.DataFrame()
            
    total_time0 = (len(move_array0) - 1) / 10
    #plot_path_in_game(move_array0,total_time0,0,60,-40,40)
    
    total_time1 = (len(move_array1) - 1) / 10
    #plot_path_in_game(move_array1,total_time1,0,60,-40,40)
    
    total_time2 = (len(move_array2) - 1) / 10
    #plot_path_in_game(move_array2,total_time2,0,60,-40,40)
    
    #total_time3= (len(move_array3) - 1) / 10
    #plot_path_in_game(move_array3,total_time3,22,60,20,53)


    df1 = pd.DataFrame(move_array1, index =[i for i in range(len(move_array1))],columns =['gameKey', 'playId', 'playerId', 'ts', 'x', 'y', 's', 'a', 'event','route', 'acc_x', 'acc_y', 'left_or_right_route', 'inverted_x','inverted_y', 'inverted_vel_x', 'inverted_vel_y', 'inverted_acc_x','inverted_acc_y','direction'])
    df2 = pd.DataFrame(move_array2, index =[i for i in range(len(move_array2))],columns =['gameKey', 'playId', 'playerId', 'ts', 'x', 'y', 's', 'a', 'event','route', 'acc_x', 'acc_y', 'left_or_right_route', 'inverted_x','inverted_y', 'inverted_vel_x', 'inverted_vel_y', 'inverted_acc_x','inverted_acc_y','direction'])
    #df3 = pd.DataFrame(move_array3, index =[i for i in range(len(move_array3))],columns =['playerId', 'timeStamp', 'posX', 'posY', 'velX','velY', 'accX', 'accY', 'acc','vel','firstName','lastName','Group','position','totalTime'])
    df1['segment'] = 'first'
    df2['segment'] = 'second'
    #df3['segment'] = 'third'
    #print(df1)
    final = df1.append(df2)
    #final = final.append(df3)
    return final

In [23]:
players = out_df.playerId.drop_duplicates().tolist()
out_segmented = pd.DataFrame(columns =['gameKey', 'playId', 'playerId', 'ts', 'x', 'y', 's', 'a', 'event','route', 'acc_x', 'acc_y', 'left_or_right_route', 'inverted_x','inverted_y', 'inverted_vel_x', 'inverted_vel_y', 'inverted_acc_x','inverted_acc_y','direction','segment'])
for player in players:
    curr_df = out_df.loc[out_df.playerId == player]
    playIds = curr_df.playId.drop_duplicates().tolist()
    for play in playIds:
        out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))

# path to file location in github
path = r"C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\analysis\data\correlation\out_route_segmented.csv"
out_segmented.to_csv(path)

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 

...

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = df1.append(df2)
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\1460607350.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_segmented = out_segmented.append(separate_segments_out_route(curr_df, play))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_player_df['direction'] = direction
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\3420404441.py:88: FutureWarning: The frame

In [24]:
out_segmented_route_left = out_segmented.loc[out_segmented.direction == 'left']
out_segmented_route_right = out_segmented.loc[out_segmented.direction == 'right']

In [25]:
players_right = out_segmented_route_right['playerId'].drop_duplicates().tolist()
columns  = ['player_id', 'route','vel_x_straight', 'acc_x_straight', 'time_straight', 'max_vel_x_straight', 'pos_vel_x_straight', 'max_acc_x_straight', 'pos_acc_x_straight', 'mean_y_straight', 'variance_y_straight','vel_x_avg_turn_right', 'vel_y_avg_turn_right', 'vel_avg_turn_right', 'acc_x_avg_turn_right','acc_y_avg_turn_right','acc_avg_turn_right','vel_x_enter_turn_right', 'vel_y_enter_turn_right', 'vel_enter_turn_right', 'acc_x_enter_turn_right', 'acc_y_enter_turn_right', 'acc_enter_turn_right', 'vel_x_end_turn_right', 'vel_y_end_turn_right','vel_end_turn_right','acc_x_end_turn_right', 'acc_y_end_turn_right','acc_end_turn_right', 'change_x_turn_right', 'change_y_turn_right', 'change_pos_turn_right']
out_route_metrics_right = pd.DataFrame(columns)
for player in players_right:
    curr_df = pd.DataFrame(columns = columns)
    player_df = out_segmented_route_right.loc[out_segmented_route_right.playerId == player]
    playIds = player_df['playId'].drop_duplicates().tolist()
    for play in playIds:
        curr_df.loc[len(curr_df)] = metric_combiner(player_df,play)
    curr_df['route'] = 'out'
    out_route_metrics_right = out_route_metrics_right.append(percentile_metrics(curr_df,columns ))

columns = ['player_id', 'route','vel_x_straight', 'acc_x_straight', 'time_straight', 'max_vel_x_straight', 'pos_vel_x_straight', 'max_acc_x_straight', 'pos_acc_x_straight', 'mean_y_straight', 'variance_y_straight','vel_x_avg_turn_left', 'vel_y_avg_turn_left', 'vel_avg_turn_left', 'acc_x_avg_turn_left','acc_y_avg_turn_left','acc_avg_turn_left','vel_x_enter_turn_left', 'vel_y_enter_turn_left', 'vel_enter_turn_left', 'acc_x_enter_turn_left', 'acc_y_enter_turn_left', 'acc_enter_turn_left', 'vel_x_end_turn_left', 'vel_y_end_turn_left','vel_end_turn_left','acc_x_end_turn_left', 'acc_y_end_turn_left','acc_end_turn_left', 'change_x_turn_left', 'change_y_turn_left', 'change_pos_turn_left']
players_left = out_segmented_route_left['playerId'].drop_duplicates().tolist()
out_route_metrics_left = pd.DataFrame(columns = columns)
for player in players_left:
    curr_df = pd.DataFrame(columns = columns)
    player_df = out_segmented_route_left.loc[out_segmented_route_left.playerId == player]
    playIds = player_df['playId'].drop_duplicates().tolist()
    for play in playIds:
        curr_df.loc[len(curr_df)] = metric_combiner(player_df,play)
    curr_df['route'] = 'out'
    out_route_metrics_left = out_route_metrics_left.append(percentile_metrics(curr_df,columns))
    

out_route_metrics_left

vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_right
vel_y_avg_turn_right
vel_avg_turn_right
acc_x_avg_turn_right
acc_y_avg_turn_right
acc_avg_turn_right
vel_x_enter_turn_right
vel_y_enter_turn_right
vel_enter_turn_right
acc_x_enter_turn_right
acc_y_enter_turn_right
acc_enter_turn_right
vel_x_end_turn_right
vel_y_end_turn_right
vel_end_turn_right
acc_x_end_turn_right
acc_y_end_turn_right
acc_end_turn_right
change_x_turn_right
change_y_turn_right
change_pos_turn_right


C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\192977427.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_route_metrics_right = out_route_metrics_right.append(percentile_metrics(curr_df,columns ))


vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_right
vel_y_avg_turn_right
vel_avg_turn_right
acc_x_avg_turn_right
acc_y_avg_turn_right
acc_avg_turn_right
vel_x_enter_turn_right
vel_y_enter_turn_right
vel_enter_turn_right
acc_x_enter_turn_right
acc_y_enter_turn_right
acc_enter_turn_right
vel_x_end_turn_right
vel_y_end_turn_right
vel_end_turn_right
acc_x_end_turn_right
acc_y_end_turn_right
acc_end_turn_right
change_x_turn_right
change_y_turn_right
change_pos_turn_right


C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\192977427.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_route_metrics_right = out_route_metrics_right.append(percentile_metrics(curr_df,columns ))


vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_right
vel_y_avg_turn_right
vel_avg_turn_right
acc_x_avg_turn_right
acc_y_avg_turn_right
acc_avg_turn_right
vel_x_enter_turn_right
vel_y_enter_turn_right
vel_enter_turn_right
acc_x_enter_turn_right
acc_y_enter_turn_right
acc_enter_turn_right
vel_x_end_turn_right
vel_y_end_turn_right
vel_end_turn_right
acc_x_end_turn_right
acc_y_end_turn_right
acc_end_turn_right
change_x_turn_right
change_y_turn_right
change_pos_turn_right
vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_right
vel_y_avg_turn_right
vel_avg_turn_right
acc_x_avg_turn_right
acc_y_avg_turn_right
acc_avg_turn_right
vel_x_enter_turn_right
vel_y_enter_turn_right
vel_enter_turn_right
acc_x_enter_turn_right
acc_y_enter_turn_right
acc_enter_turn_right
vel_x_end_turn_right

...

vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_left
vel_y_avg_turn_left
vel_avg_turn_left
acc_x_avg_turn_left
acc_y_avg_turn_left
acc_avg_turn_left
vel_x_enter_turn_left
vel_y_enter_turn_left
vel_enter_turn_left
acc_x_enter_turn_left
acc_y_enter_turn_left
acc_enter_turn_left
vel_x_end_turn_left
vel_y_end_turn_left
vel_end_turn_left
acc_x_end_turn_left
acc_y_end_turn_left
acc_end_turn_left
change_x_turn_left
change_y_turn_left
change_pos_turn_left
vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_left
vel_y_avg_turn_left
vel_avg_turn_left
acc_x_avg_turn_left
acc_y_avg_turn_left
acc_avg_turn_left
vel_x_enter_turn_left
vel_y_enter_turn_left
vel_enter_turn_left
acc_x_enter_turn_left
acc_y_enter_turn_left
acc_enter_turn_left
vel_x_end_turn_left
vel_y_end_turn_left
vel_end_turn_

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\192977427.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_route_metrics_left = out_route_metrics_left.append(percentile_metrics(curr_df,columns))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\192977427.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_route_metrics_left = out_route_metrics_left.append(percentile_metrics(curr_df,columns))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\192977427.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_route_metrics_left = out_route_metrics_left.append(percentile_metrics(curr_df,columns))


vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_left
vel_y_avg_turn_left
vel_avg_turn_left
acc_x_avg_turn_left
acc_y_avg_turn_left
acc_avg_turn_left
vel_x_enter_turn_left
vel_y_enter_turn_left
vel_enter_turn_left
acc_x_enter_turn_left
acc_y_enter_turn_left
acc_enter_turn_left
vel_x_end_turn_left
vel_y_end_turn_left
vel_end_turn_left
acc_x_end_turn_left
acc_y_end_turn_left
acc_end_turn_left
change_x_turn_left
change_y_turn_left
change_pos_turn_left
vel_x_straight
acc_x_straight
max_vel_x_straight
pos_vel_x_straight
max_acc_x_straight
pos_acc_x_straight
mean_y_straight
variance_y_straight
vel_x_avg_turn_left
vel_y_avg_turn_left
vel_avg_turn_left
acc_x_avg_turn_left
acc_y_avg_turn_left
acc_avg_turn_left
vel_x_enter_turn_left
vel_y_enter_turn_left
vel_enter_turn_left
acc_x_enter_turn_left
acc_y_enter_turn_left
acc_enter_turn_left
vel_x_end_turn_left
vel_y_end_turn_left
vel_end_turn_

C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\192977427.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_route_metrics_left = out_route_metrics_left.append(percentile_metrics(curr_df,columns))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\192977427.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_route_metrics_left = out_route_metrics_left.append(percentile_metrics(curr_df,columns))
C:\Users\mattd\AppData\Local\Temp\ipykernel_39500\192977427.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_route_metrics_left = out_route_metrics_left.append(percentile_metrics(curr_df,columns))


,player_id,route,vel_x_straight,acc_x_straight,time_straight,max_vel_x_straight,pos_vel_x_straight,max_acc_x_straight,pos_acc_x_straight,mean_y_straight,...,acc_enter_turn_left,vel_x_end_turn_left,vel_y_end_turn_left,vel_end_turn_left,acc_x_end_turn_left,acc_y_end_turn_left,acc_end_turn_left,change_x_turn_left,change_y_turn_left,change_pos_turn_left
0,54476,out,4.433676,4.169397,1.900000,7.766800,9.484819,10.409615,3.104430,2.501576,...,6.740871,3.664879,5.986759,6.463021,4.788052,1.249748,5.310026,6.264297,11.662057,12.858167
0,54517,out,4.726173,3.924114,1.625000,7.672854,10.950804,7.613841,0.789054,0.651310,...,3.848446,1.537893,5.236860,5.128468,2.014526,-1.817229,6.713500,7.322676,15.294014,16.573147
0,54475,out,4.691607,4.181021,1.725000,7.868157,10.241516,11.102188,2.218806,1.552901,...,5.309272,1.553320,6.418057,6.568620,2.698421,-0.181780,4.310748,7.844255,16.102975,17.536611
0,54481,out,4.317628,4.126970,1.888889,8.086244,9.567829,10.784572,2.151946,1.609803,...,7.029476,2.483411,7.493618,7.717713,1.960345,1.995808,4.983385,5.224623,15.394944,15.842535
0,54519,out,4.594147,4.425692,1.525000,8.551343,8.830547,8.542438,0.584637,0.544534,...,3.616284,4.672376,7.724163,8.434467,-0.355146,2.326174,2.593923,6.739484,7.996427,9.696603
0,54499,out,2.784308,3.801429,2.566667,7.112553,12.637013,8.108725,1.169646,6.027626,...,5.086490,1.519155,5.932243,5.882570,-1.000000,0.644854,2.568269,12.092782,23.945523,26.807356
0,54628,out,4.126386,3.944164,1.400000,6.950845,6.046129,7.442058,0.849785,0.321121,...,6.752173,2.197852,8.556644,8.470221,5.718160,1.983863,6.041997,3.308894,12.521437,12.904270
0,54564,out,3.829237,3.644354,1.614286,6.966761,8.265041,8.828788,1.295370,0.903835,...,4.871259,2.028839,5.723071,5.801139,3.470533,0.585941,4.824919,5.452266,12.558229,13.092734
0,54613,out,4.223148,4.546717,2.083333,7.636599,9.347829,9.829119,1.004416,1.306528,...,3.773129,3.123788,6.790674,6.816413,2.812159,6.121618,6.761434,4.484137,15.934531,15.968913
0,54483,out,4.525469,3.920012,1.725000,7.846514,9.617377,9.113841,0.954861,0.614551,...,3.620448,4.016808,6.303531,6.865042,1.811213,2.496332,4.411129,5.567050,12.669814,12.807636


In [26]:
# paths to the file location in the github
#path_right = 'in_game/data/in_route_metrics_right.csv'
#path_left = 'in_game/data/in_route_metrics_left.csv'
path_left = r"C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\analysis\data\correlation\out_route_metrics_left.csv"
path_right = r"C:\Users\mattd\Documents\GitHub\2022NFLScoutingCombineAnalysis\analysis\data\correlation\out_route_metrics_right.csv"
out_route_metrics_right.to_csv(path_right)
out_route_metrics_left.to_csv(path_left)